In [1]:
library(tidyverse)
library(reshape2)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.4     ✔ dplyr   1.0.2
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()


Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths




In [2]:
library("RSQLite")

## connect to db
con <- dbConnect(drv=RSQLite::SQLite(), dbname="../input/soccer/database.sqlite")

## list all tables
tables <- dbListTables(con)

## exclude sqlite_sequence (contains table information)
tables <- tables[tables != "sqlite_sequence"]

lDataFrames <- vector("list", length=length(tables))

## create a data.frame for each table
for (i in seq(along=tables)) {
  lDataFrames[[i]] <- dbGetQuery(conn=con, statement=paste("SELECT * FROM '", tables[[i]], "'", sep=""))
}



In [3]:
match <- data.frame(lDataFrames[3])
team <- data.frame(lDataFrames[6])
# head(match,2)
# head(team,2)


In [4]:
match <- match %>% select(match_api_id,date,home_team_api_id,away_team_api_id,home_team_goal, away_team_goal)
team <- team %>% select(team_api_id,team_long_name)


In [5]:
match$home_score <- (match$home_team_goal - match$away_team_goal)
match$away_score <- - match$home_score
match$home_win_rate <- as.integer(as.logical(match$home_score>0))
match$away_win_rate <- as.integer(as.logical(match$away_score>0))
head(match,6)


,match_api_id,date,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_score,away_score,home_win_rate,away_win_rate
,<int>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,492473,2008-08-17 00:00:00,9987,9993,1,1,0,0,0,0
2,492474,2008-08-16 00:00:00,10000,9994,0,0,0,0,0,0
3,492475,2008-08-16 00:00:00,9984,8635,0,3,-3,3,0,1
4,492476,2008-08-17 00:00:00,9991,9998,5,0,5,-5,1,0
5,492477,2008-08-16 00:00:00,7947,9985,1,3,-2,2,0,1
6,492478,2008-09-24 00:00:00,8203,8342,1,1,0,0,0,0


In [6]:
match_home <- match %>% select(match_api_id,date,home_team_api_id,home_score,home_win_rate) %>% 
  rename(
    team_api_id = home_team_api_id,
    score = home_score,
    win_rate = home_win_rate
    )
match_away <- match %>% select(match_api_id,date,away_team_api_id,away_score, away_win_rate) %>% 
  rename(
    team_api_id = away_team_api_id,
    score = away_score,
    win_rate = away_win_rate
    )

In [7]:
total <- rbind(match_home, match_away)
total$date <- as.Date(total$date)
head(arrange(total,match_api_id))

,match_api_id,date,team_api_id,score,win_rate
,<int>,<date>,<int>,<int>,<int>
1,483129,2008-08-09,8583,1,1
2,483129,2008-08-09,9830,-1,0
3,483130,2008-08-09,9827,1,1
4,483130,2008-08-09,7819,-1,0
5,483131,2008-08-09,9746,1,1
6,483131,2008-08-09,9831,-1,0


In [8]:
match14 = with(total, total[(date >= "2014-01-01" & date < "2015-01-01"),])
match15 = with(total, total[(date >= "2015-01-01" & date < "2016-01-01"),])
                            

In [9]:
df14 = subset(match14, select=-c(date))
df15 = subset(match15, select=-c(date))

In [10]:
team_ranking14_byscore <- aggregate(df14$score, by=list(team_api_id=df14$team_api_id), FUN=mean) %>% rename(score=x)
team_ranking15_byscore <- aggregate(df15$score, by=list(team_api_id=df15$team_api_id), FUN=mean) %>% rename(score=x)

In [11]:
team_ranking14_bywin <- aggregate(df14$win_rate, by=list(team_api_id=df14$team_api_id), FUN=mean) %>% rename(win_rate=x)
team_ranking15_bywin <- aggregate(df15$win_rate, by=list(team_api_id=df15$team_api_id), FUN=mean) %>% rename(win_rate=x)

In [12]:
team_ranking14 <- merge(team,team_ranking14_byscore, by="team_api_id",all.team_ranking14_byscore = TRUE)
team_ranking15 <- merge(team,team_ranking15_byscore,by="team_api_id",all.team_ranking14_byscore = TRUE)

team_ranking14 <- arrange(merge(team_ranking14,team_ranking14_bywin, by="team_api_id"),-win_rate)
team_ranking15 <- arrange(merge(team_ranking15,team_ranking15_bywin,by="team_api_id"),-win_rate)




In [13]:
head(team_ranking14)
head(team_ranking15)

,team_api_id,team_long_name,score,win_rate
,<int>,<chr>,<dbl>,<dbl>
1,9772,SL Benfica,1.633333,0.8333333
2,9823,FC Bayern Munich,2.114286,0.8285714
3,9885,Juventus,1.513514,0.8108108
4,8640,PSV,1.181818,0.7575758
5,8633,Real Madrid CF,2.222222,0.7500000
6,9925,Celtic,1.868421,0.7368421


,team_api_id,team_long_name,score,win_rate
,<int>,<chr>,<dbl>,<dbl>
1,9773,FC Porto,1.764706,0.7941176
2,8634,FC Barcelona,2.105263,0.7894737
3,9847,Paris Saint-Germain,1.763158,0.7894737
4,9925,Celtic,1.948718,0.7692308
5,8640,PSV,1.470588,0.7647059
6,9768,Sporting CP,1.205882,0.7647059
